<a href="https://colab.research.google.com/github/YichengZou626/COMP590_intro_to_deep_learning/blob/main/Homework_8(Yicheng_Zou).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 8

In this homework, you will be training a sequence-to-sequence model. Unfortunately, these models can be somewhat expensive to train, and the free GPU/TPU from colab is not sufficient to train a sequence-to-sequence model on a terribly large/difficult task. So, you will be translating short phrases from English to French using a small dataset comes from [Tatoeba](tatoeba.org).

This homework contains example mxnet code that mainly comes from the textbook. There is an analogous Pytorch example in the textbook you can use instead.

In [ ]:
!pip install d2l mxnet

     |████████████████████████████████| 82 kB 461 kB/s 
     |████████████████████████████████| 47.3 MB 1.2 MB/s 
     |████████████████████████████████| 61 kB 9.8 MB/s 
     |████████████████████████████████| 82 kB 866 kB/s 
     |████████████████████████████████| 11.6 MB 23.0 MB/s 
     |████████████████████████████████| 9.9 MB 26.3 MB/s 
     |████████████████████████████████| 20.1 MB 68.0 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Succes

In [ ]:
import collections
from d2l import mxnet as d2l
import math
from mxnet import np, npx, init, gluon, autograd
from mxnet.gluon import nn, rnn
npx.set_np()

# Example sequence-to-sequence model implementation

In [ ]:
class Seq2SeqEncoder(d2l.Encoder):
    """The RNN encoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # Embedding layer for looking up embeddings of tokens
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=dropout)

    def forward(self, X, *args):
        # The output `X` shape: (`batch_size`, `num_steps`, `embed_size`)
        X = self.embedding(X)
        # In RNN models, the first axis corresponds to time steps
        X = X.swapaxes(0, 1)
        state = self.rnn.begin_state(batch_size=X.shape[1], ctx=X.ctx)
        output, state = self.rnn(X, state)
        # `output` shape: (`num_steps`, `batch_size`, `num_hiddens`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class Seq2SeqDecoder(d2l.Decoder):
    """The RNN decoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=dropout)
        self.dense = nn.Dense(vocab_size, flatten=False)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # The output `X` shape: (`num_steps`, `batch_size`, `embed_size`)
        X = self.embedding(X).swapaxes(0, 1)
        output, state = self.rnn(X, state)
        output = self.dense(output).swapaxes(0, 1)
        # `output` shape: (`batch_size`, `num_steps`, `vocab_size`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class MaskedSoftmaxCELoss(gluon.loss.SoftmaxCELoss):
    """The softmax cross-entropy loss with masks."""
    # `pred` shape: (`batch_size`, `num_steps`, `vocab_size`)
    # `label` shape: (`batch_size`, `num_steps`)
    # `valid_len` shape: (`batch_size`,)
    def forward(self, pred, label, valid_len):
        # `weights` shape: (`batch_size`, `num_steps`, 1)
        weights = np.expand_dims(np.ones_like(label), axis=-1)
        # sequence_mask zeros out entries that are beyond the sequence lengths
        weights = npx.sequence_mask(weights, valid_len, True, axis=1)
        return super(MaskedSoftmaxCELoss, self).forward(pred, label, weights)

def train_seq2seq(net, data_iter, lr, num_epochs, tgt_vocab, device):
    """Train a model for sequence to sequence."""
    net.initialize(init.Xavier(), force_reinit=True, ctx=device)
    trainer = gluon.Trainer(net.collect_params(), 'adam',
                            {'learning_rate': lr})
    loss = MaskedSoftmaxCELoss()
    for epoch in range(num_epochs):
        timer = d2l.Timer()
        metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
        for batch in data_iter:
            X, X_valid_len, Y, Y_valid_len = [
                x.as_in_ctx(device) for x in batch]
            bos = np.array(
                [tgt_vocab['<bos>']] * Y.shape[0], ctx=device).reshape(-1, 1)
            dec_input = np.concatenate([bos, Y[:, :-1]], 1)  # Teacher forcing
            with autograd.record():
                Y_hat, _ = net(X, dec_input, X_valid_len)
                l = loss(Y_hat, Y, Y_valid_len)
            l.backward()
            # Add gradient clipping
            d2l.grad_clipping(net, 1)
            num_tokens = Y_valid_len.sum()
            trainer.step(num_tokens)
            metric.add(l.sum(), num_tokens)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, loss {(metric[0] / metric[1])}")
    print(f'loss {metric[0] / metric[1]:.3f}, {metric[1] / timer.stop():.1f} '
          f'tokens/sec on {str(device)}')
    
def predict_seq2seq(net, src_sentence, src_vocab, tgt_vocab, num_steps, device):
    """Predict for sequence to sequence."""
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [src_vocab['<eos>']]
    enc_valid_len = np.array([len(src_tokens)], ctx=device)
    src_tokens = d2l.truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    # Add the batch axis
    enc_X = np.expand_dims(np.array(src_tokens, ctx=device), axis=0)
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    # Add the batch axis
    dec_X = np.expand_dims(np.array([tgt_vocab['<bos>']], ctx=device), axis=0)
    output_seq = []
    for _ in range(num_steps):
        Y, dec_state = net.decoder(dec_X, dec_state)
        # We use the token with the highest prediction likelihood as the input
        # of the decoder at the next time step
        dec_X = Y.argmax(axis=2)
        pred = dec_X.squeeze(axis=0).astype('int32').item()
        # Once the end-of-sequence token is predicted, the generation of the
        # output sequence is complete
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq)),

embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 300, d2l.try_gpu()

train_iter, src_vocab, tgt_vocab = d2l.load_data_nmt(batch_size, num_steps)
encoder = Seq2SeqEncoder(len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqDecoder(len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)
train_seq2seq(net, train_iter, lr, num_epochs, tgt_vocab, device)

# Generate some sequences!
engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
for eng in engs:
    translation = predict_seq2seq(net, eng, src_vocab, tgt_vocab, num_steps, device)
    print(f'{eng} => {translation}')

Epoch 10, loss 0.24009968360921896
Epoch 20, loss 0.18821761046090213
Epoch 30, loss 0.14967849788493126
Epoch 40, loss 0.12620719142524991
Epoch 50, loss 0.10735590933627474
Epoch 60, loss 0.09367445818508108
Epoch 70, loss 0.08294192039793
Epoch 80, loss 0.07240383046107797
Epoch 90, loss 0.06525398980252906
Epoch 100, loss 0.05817260652112004
Epoch 110, loss 0.05277993950211406
Epoch 120, loss 0.04799309512472021
Epoch 130, loss 0.04519115288114041
Epoch 140, loss 0.04099128115885561
Epoch 150, loss 0.03808096495729316
Epoch 160, loss 0.03537184964104555
Epoch 170, loss 0.03259612011749976
Epoch 180, loss 0.031448045803401846
Epoch 190, loss 0.029107022595095945
Epoch 200, loss 0.027584049655714638
Epoch 210, loss 0.02656651359522075
Epoch 220, loss 0.024949619968028483
Epoch 230, loss 0.02403260515881823
Epoch 240, loss 0.024495033427797307
Epoch 250, loss 0.02257408667905553
Epoch 260, loss 0.02182258625098074
Epoch 270, loss 0.021082518655265798
Epoch 280, loss 0.0211191771303467

# Problems

1. In the example, sequence-to-sequence learning is enabled by passing the final encoder state to the decoder and using it to initialize the decoder state. In class, we also discussed passing the encoder state to the decoder by concatenating it with the embeddings used as input to the decoder. Implement this. Does this change the loss and/or the predictions of the model? Why do you think this is?
1. The prediction function above uses greedy sampling (choosing the most probable token at each time step). Change it so that it instead samples randomly at each time step according to the predicted distribution. Generate the predictions a few times for the example English sentences provided. Do they ever change? Why do you think they do or don't change? Hint: You will need to implement a categorical sampler. One way to do this with numpy is described [here](https://stackoverflow.com/a/62875642).
1. In class, we discussed deep RNNs, which consist of stacks of RNN layers. Change the model so that it has two GRU layers in the encoder and two GRU layers in the decoder. Is training faster or slower? Is the final loss higher or lower? Hint: It might help you to use [SequentialRNNCell](https://mxnet.apache.org/versions/1.6/api/python/docs/api/gluon/rnn/index.html#mxnet.gluon.rnn.SequentialRNNCell).

# 1: Concatenating the encoder state with embedding 

In [ ]:
class Seq2SeqEncoder1(d2l.Encoder):
    """The RNN encoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder1, self).__init__(**kwargs)
        # Embedding layer for looking up embeddings of tokens
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=dropout)

    def forward(self, X, *args):
        # The output `X` shape: (`batch_size`, `num_steps`, `embed_size`)
        X = self.embedding(X)
        # In RNN models, the first axis corresponds to time steps
        X = X.swapaxes(0, 1)
        state = self.rnn.begin_state(batch_size=X.shape[1], ctx=X.ctx)
        output, state = self.rnn(X, state)
        # `output` shape: (`num_steps`, `batch_size`, `num_hiddens`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class Seq2SeqDecoder1(d2l.Decoder):
    """The RNN decoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder1, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=dropout)
        self.dense = nn.Dense(vocab_size, flatten=False)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # The output `X` shape: (`num_steps`, `batch_size`, `embed_size`)
        X = self.embedding(X).swapaxes(0, 1)
        # `context` shape: (`batch_size`, `num_hiddens`)
        context = state[0][-1]
        # Broadcast `context` so it has the same `num_steps` as `X`
        context = np.broadcast_to(context, (
            X.shape[0], context.shape[0], context.shape[1]))
        X_and_context = np.concatenate((X, context), 2)
        output, state = self.rnn(X_and_context, state)
        output = self.dense(output).swapaxes(0, 1)
        # `output` shape: (`batch_size`, `num_steps`, `vocab_size`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state


class MaskedSoftmaxCELoss(gluon.loss.SoftmaxCELoss):
    """The softmax cross-entropy loss with masks."""
    # `pred` shape: (`batch_size`, `num_steps`, `vocab_size`)
    # `label` shape: (`batch_size`, `num_steps`)
    # `valid_len` shape: (`batch_size`,)
    def forward(self, pred, label, valid_len):
        # `weights` shape: (`batch_size`, `num_steps`, 1)
        weights = np.expand_dims(np.ones_like(label), axis=-1)
        # sequence_mask zeros out entries that are beyond the sequence lengths
        weights = npx.sequence_mask(weights, valid_len, True, axis=1)
        return super(MaskedSoftmaxCELoss, self).forward(pred, label, weights)

def train_seq2seq1(net, data_iter, lr, num_epochs, tgt_vocab, device):
    """Train a model for sequence to sequence."""
    net.initialize(init.Xavier(), force_reinit=True, ctx=device)
    trainer = gluon.Trainer(net.collect_params(), 'adam',
                            {'learning_rate': lr})
    loss = MaskedSoftmaxCELoss()
    for epoch in range(num_epochs):
        timer = d2l.Timer()
        metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
        for batch in data_iter:
            X, X_valid_len, Y, Y_valid_len = [
                x.as_in_ctx(device) for x in batch]
            bos = np.array(
                [tgt_vocab['<bos>']] * Y.shape[0], ctx=device).reshape(-1, 1)
            dec_input = np.concatenate([bos, Y[:, :-1]], 1)  # Teacher forcing
            with autograd.record():
                Y_hat, _ = net(X, dec_input, X_valid_len)
                l = loss(Y_hat, Y, Y_valid_len)
            l.backward()
            # Add gradient clipping
            d2l.grad_clipping(net, 1)
            num_tokens = Y_valid_len.sum()
            trainer.step(num_tokens)
            metric.add(l.sum(), num_tokens)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, loss {(metric[0] / metric[1])}")
    print(f'loss {metric[0] / metric[1]:.3f}, {metric[1] / timer.stop():.1f} '
          f'tokens/sec on {str(device)}')
    
def predict_seq2seq1(net, src_sentence, src_vocab, tgt_vocab, num_steps, device):
    """Predict for sequence to sequence."""
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [src_vocab['<eos>']]
    enc_valid_len = np.array([len(src_tokens)], ctx=device)
    src_tokens = d2l.truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    # Add the batch axis
    enc_X = np.expand_dims(np.array(src_tokens, ctx=device), axis=0)
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    # Add the batch axis
    dec_X = np.expand_dims(np.array([tgt_vocab['<bos>']], ctx=device), axis=0)
    output_seq = []
    for _ in range(num_steps):
        Y, dec_state = net.decoder(dec_X, dec_state)
        # We use the token with the highest prediction likelihood as the input
        # of the decoder at the next time step
        dec_X = Y.argmax(axis=2)
        pred = dec_X.squeeze(axis=0).astype('int32').item()
        # Once the end-of-sequence token is predicted, the generation of the
        # output sequence is complete
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq)),

embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 300, d2l.try_gpu()

train_iter, src_vocab, tgt_vocab = d2l.load_data_nmt(batch_size, num_steps)
encoder = Seq2SeqEncoder1(len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqDecoder1(len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)
train_seq2seq1(net, train_iter, lr, num_epochs, tgt_vocab, device)

# Generate some sequences!
engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
for eng in engs:
    translation = predict_seq2seq1(net, eng, src_vocab, tgt_vocab, num_steps, device)
    print(f'{eng} => {translation}')

Epoch 10, loss 0.23776725669959922
Epoch 20, loss 0.19640280007846897
Epoch 30, loss 0.16760551267981952
Epoch 40, loss 0.143198858889385
Epoch 50, loss 0.1261709818263919
Epoch 60, loss 0.10868061062671311
Epoch 70, loss 0.09596645696010236
Epoch 80, loss 0.08349801029383971
Epoch 90, loss 0.07486014045255947
Epoch 100, loss 0.06805979178112259
Epoch 110, loss 0.06097593010996609
Epoch 120, loss 0.05507651223577134
Epoch 130, loss 0.04916465362194531
Epoch 140, loss 0.04519039924948856
Epoch 150, loss 0.040913800281222906
Epoch 160, loss 0.03818250987906008
Epoch 170, loss 0.03571809759669208
Epoch 180, loss 0.03314604706672269
Epoch 190, loss 0.030880573648770275
Epoch 200, loss 0.029393817251566496
Epoch 210, loss 0.02757653651487072
Epoch 220, loss 0.026061763917306708
Epoch 230, loss 0.025329366906896814
Epoch 240, loss 0.02470295600936151
Epoch 250, loss 0.02391478171830851
Epoch 260, loss 0.023320843693967037
Epoch 270, loss 0.023095194254743448
Epoch 280, loss 0.022163588716401

The loss does not change yet, and the reason for that might be this task is really simple, and the encoder state and input X might cover the same information.

# 2: Using random selection for prediction

In [ ]:
class Seq2SeqEncoder2(d2l.Encoder):
    """The RNN encoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder2, self).__init__(**kwargs)
        # Embedding layer for looking up embeddings of tokens
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=dropout)

    def forward(self, X, *args):
        # The output `X` shape: (`batch_size`, `num_steps`, `embed_size`)
        X = self.embedding(X)
        # In RNN models, the first axis corresponds to time steps
        X = X.swapaxes(0, 1)
        state = self.rnn.begin_state(batch_size=X.shape[1], ctx=X.ctx)
        output, state = self.rnn(X, state)
        # `output` shape: (`num_steps`, `batch_size`, `num_hiddens`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class Seq2SeqDecoder2(d2l.Decoder):
    """The RNN decoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder2, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=dropout)
        self.dense = nn.Dense(vocab_size, flatten=False)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # The output `X` shape: (`num_steps`, `batch_size`, `embed_size`)
        X = self.embedding(X).swapaxes(0, 1)
        output, state = self.rnn(X, state)
        output = self.dense(output).swapaxes(0, 1)
        # `output` shape: (`batch_size`, `num_steps`, `vocab_size`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class MaskedSoftmaxCELoss(gluon.loss.SoftmaxCELoss):
    """The softmax cross-entropy loss with masks."""
    # `pred` shape: (`batch_size`, `num_steps`, `vocab_size`)
    # `label` shape: (`batch_size`, `num_steps`)
    # `valid_len` shape: (`batch_size`,)
    def forward(self, pred, label, valid_len):
        # `weights` shape: (`batch_size`, `num_steps`, 1)
        weights = np.expand_dims(np.ones_like(label), axis=-1)
        # sequence_mask zeros out entries that are beyond the sequence lengths
        weights = npx.sequence_mask(weights, valid_len, True, axis=1)
        return super(MaskedSoftmaxCELoss, self).forward(pred, label, weights)

def train_seq2seq2(net, data_iter, lr, num_epochs, tgt_vocab, device):
    """Train a model for sequence to sequence."""
    net.initialize(init.Xavier(), force_reinit=True, ctx=device)
    trainer = gluon.Trainer(net.collect_params(), 'adam',
                            {'learning_rate': lr})
    loss = MaskedSoftmaxCELoss()
    for epoch in range(num_epochs):
        timer = d2l.Timer()
        metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
        for batch in data_iter:
            X, X_valid_len, Y, Y_valid_len = [
                x.as_in_ctx(device) for x in batch]
            bos = np.array(
                [tgt_vocab['<bos>']] * Y.shape[0], ctx=device).reshape(-1, 1)
            dec_input = np.concatenate([bos, Y[:, :-1]], 1)  # Teacher forcing
            with autograd.record():
                Y_hat, _ = net(X, dec_input, X_valid_len)
                l = loss(Y_hat, Y, Y_valid_len)
            l.backward()
            # Add gradient clipping
            d2l.grad_clipping(net, 1)
            num_tokens = Y_valid_len.sum()
            trainer.step(num_tokens)
            metric.add(l.sum(), num_tokens)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, loss {(metric[0] / metric[1])}")
    print(f'loss {metric[0] / metric[1]:.3f}, {metric[1] / timer.stop():.1f} '
          f'tokens/sec on {str(device)}')


embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 300, d2l.try_gpu()

train_iter, src_vocab, tgt_vocab = d2l.load_data_nmt(batch_size, num_steps)
encoder = Seq2SeqEncoder2(len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqDecoder2(len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)
train_seq2seq2(net, train_iter, lr, num_epochs, tgt_vocab, device)


Epoch 10, loss 0.23005600756240238
Epoch 20, loss 0.1829521384871602
Epoch 30, loss 0.15010172961406565
Epoch 40, loss 0.1245324319856742
Epoch 50, loss 0.10314159813662863
Epoch 60, loss 0.09026079541911576
Epoch 70, loss 0.07599551693654727
Epoch 80, loss 0.06703110596928152
Epoch 90, loss 0.05804091351841998
Epoch 100, loss 0.05135159546913288
Epoch 110, loss 0.0447427133367269
Epoch 120, loss 0.04036101108927841
Epoch 130, loss 0.03549119441915525
Epoch 140, loss 0.03346643256465547
Epoch 150, loss 0.029964339306203458
Epoch 160, loss 0.028287501937523923
Epoch 170, loss 0.026401704523986554
Epoch 180, loss 0.025101357958815032
Epoch 190, loss 0.023937299935567195
Epoch 200, loss 0.023435795039747794
Epoch 210, loss 0.022183061772000832
Epoch 220, loss 0.02191745564019009
Epoch 230, loss 0.02156379775520574
Epoch 240, loss 0.021537907618042616
Epoch 250, loss 0.021729603098959775
Epoch 260, loss 0.020927067958947384
Epoch 270, loss 0.020152349398859556
Epoch 280, loss 0.01953374190

In [ ]:
def predict_seq2seq2(net, src_sentence, src_vocab, tgt_vocab, num_steps, device):
    """Predict for sequence to sequence."""
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [src_vocab['<eos>']]
    enc_valid_len = np.array([len(src_tokens)], ctx=device)
    src_tokens = d2l.truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    # Add the batch axis
    enc_X = np.expand_dims(np.array(src_tokens, ctx=device), axis=0)
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    # Add the batch axis
    dec_X = np.expand_dims(np.array([tgt_vocab['<bos>']], ctx=device), axis=0)
    output_seq = []
    for _ in range(num_steps):
        Y, dec_state = net.decoder(dec_X, dec_state)
        Y = np.exp(Y) / np.sum(np.exp(Y), axis=2)
        n1,n2,m = Y.shape
        # We use the token with the highest prediction likelihood as the input
        # of the decoder at the next time step
        cum_prob = np.cumsum(Y, axis=2) # shape (n1, n2, m)
        r = np.random.uniform(size=(n1, n2, 1))
        for i in range(m):
          p = cum_prob[:,:,i]
          if p>r:
            dec_X = np.expand_dims(np.array([i], ctx=device), axis=0)
            pred = i
            break 
        # Once the end-of-sequence token is predicted, the generation of the
        # output sequence is complete
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq)),

# Generate some sequences!
engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
for eng in engs:
    translation = predict_seq2seq2(net, eng, src_vocab, tgt_vocab, num_steps, device)
    print(f'{eng} => {translation}')

go . => ('va !',)
i lost . => ("j'ai perdu .",)
he's calm . => ('venez !',)
i'm home . => ('je suis chez moi .',)


After running several times, the predicted output does not change very much. The reason for that might be when we using the cumsum function, the first index that cumsum probability is large than a uniform random value is very similar to the max probability.

# 3: Using two layers of GRU

In [ ]:
class Seq2SeqEncoder3(d2l.Encoder):
    """The RNN encoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder3, self).__init__(**kwargs)
        # Embedding layer for looking up embeddings of tokens
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.SequentialRNNCell()
        self.rnn.add(rnn.GRU(num_hiddens, num_layers, dropout=dropout))
        self.rnn.add(rnn.GRU(num_hiddens, num_layers, dropout=dropout))



    def forward(self, X, *args):
        # The output `X` shape: (`batch_size`, `num_steps`, `embed_size`)
        X = self.embedding(X)
        # In RNN models, the first axis corresponds to time steps
        X = X.swapaxes(0, 1)
        state = self.rnn.begin_state(batch_size=X.shape[1], ctx=X.ctx)
        output, state = self.rnn(X, state)
        # `output` shape: (`num_steps`, `batch_size`, `num_hiddens`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class Seq2SeqDecoder3(d2l.Decoder):
    """The RNN decoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder3, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.SequentialRNNCell()
        self.rnn.add(rnn.GRU(num_hiddens, num_layers, dropout=dropout))
        self.rnn.add(rnn.GRU(num_hiddens, num_layers, dropout=dropout))
        self.dense = nn.Dense(vocab_size, flatten=False)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # The output `X` shape: (`num_steps`, `batch_size`, `embed_size`)
        X = self.embedding(X).swapaxes(0, 1)
        output, state = self.rnn(X, state)
        output = self.dense(output).swapaxes(0, 1)
        # `output` shape: (`batch_size`, `num_steps`, `vocab_size`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class MaskedSoftmaxCELoss(gluon.loss.SoftmaxCELoss):
    """The softmax cross-entropy loss with masks."""
    # `pred` shape: (`batch_size`, `num_steps`, `vocab_size`)
    # `label` shape: (`batch_size`, `num_steps`)
    # `valid_len` shape: (`batch_size`,)
    def forward(self, pred, label, valid_len):
        # `weights` shape: (`batch_size`, `num_steps`, 1)
        weights = np.expand_dims(np.ones_like(label), axis=-1)
        # sequence_mask zeros out entries that are beyond the sequence lengths
        weights = npx.sequence_mask(weights, valid_len, True, axis=1)
        return super(MaskedSoftmaxCELoss, self).forward(pred, label, weights)

def train_seq2seq3(net, data_iter, lr, num_epochs, tgt_vocab, device):
    """Train a model for sequence to sequence."""
    net.initialize(init.Xavier(), force_reinit=True, ctx=device)
    trainer = gluon.Trainer(net.collect_params(), 'adam',
                            {'learning_rate': lr})
    loss = MaskedSoftmaxCELoss()
    for epoch in range(num_epochs):
        timer = d2l.Timer()
        metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
        for batch in data_iter:
            X, X_valid_len, Y, Y_valid_len = [
                x.as_in_ctx(device) for x in batch]
            bos = np.array(
                [tgt_vocab['<bos>']] * Y.shape[0], ctx=device).reshape(-1, 1)
            dec_input = np.concatenate([bos, Y[:, :-1]], 1)  # Teacher forcing
            with autograd.record():
                Y_hat, _ = net(X, dec_input, X_valid_len)
                l = loss(Y_hat, Y, Y_valid_len)
            l.backward()
            # Add gradient clipping
            d2l.grad_clipping(net, 1)
            num_tokens = Y_valid_len.sum()
            trainer.step(num_tokens)
            metric.add(l.sum(), num_tokens)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, loss {(metric[0] / metric[1])}")
    print(f'loss {metric[0] / metric[1]:.3f}, {metric[1] / timer.stop():.1f} '
          f'tokens/sec on {str(device)}')
    
def predict_seq2seq3(net, src_sentence, src_vocab, tgt_vocab, num_steps, device):
    """Predict for sequence to sequence."""
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [src_vocab['<eos>']]
    enc_valid_len = np.array([len(src_tokens)], ctx=device)
    src_tokens = d2l.truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    # Add the batch axis
    enc_X = np.expand_dims(np.array(src_tokens, ctx=device), axis=0)
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    # Add the batch axis
    dec_X = np.expand_dims(np.array([tgt_vocab['<bos>']], ctx=device), axis=0)
    output_seq = []
    for _ in range(num_steps):
        Y, dec_state = net.decoder(dec_X, dec_state)
        # We use the token with the highest prediction likelihood as the input
        # of the decoder at the next time step
        dec_X = Y.argmax(axis=2)
        pred = dec_X.squeeze(axis=0).astype('int32').item()
        # Once the end-of-sequence token is predicted, the generation of the
        # output sequence is complete
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq)),

embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 300, d2l.try_gpu()

train_iter, src_vocab, tgt_vocab = d2l.load_data_nmt(batch_size, num_steps)
encoder = Seq2SeqEncoder3(len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqDecoder3(len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)
train_seq2seq3(net, train_iter, lr, num_epochs, tgt_vocab, device)

# Generate some sequences!
engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
for eng in engs:
    translation = predict_seq2seq3(net, eng, src_vocab, tgt_vocab, num_steps, device)
    print(f'{eng} => {translation}')

Epoch 10, loss 0.2503205088825969
Epoch 20, loss 0.20541966425908076
Epoch 30, loss 0.18181233666910518
Epoch 40, loss 0.1634592576460405
Epoch 50, loss 0.15013314269274908
Epoch 60, loss 0.13469427695006003
Epoch 70, loss 0.12309898630177492
Epoch 80, loss 0.11230183510928583
Epoch 90, loss 0.1052308215817238
Epoch 100, loss 0.09527665730866332
Epoch 110, loss 0.08791289616644453
Epoch 120, loss 0.07935538774210371
Epoch 130, loss 0.07287428558271544
Epoch 140, loss 0.06668792583491097
Epoch 150, loss 0.06186449860832157
Epoch 160, loss 0.058285869701975496
Epoch 170, loss 0.051888693884947504
Epoch 180, loss 0.047289984379167496
Epoch 190, loss 0.04386471425956837
Epoch 200, loss 0.040574314368141774
Epoch 210, loss 0.038813990150460664
Epoch 220, loss 0.03788162037782995
Epoch 230, loss 0.03444385838198971
Epoch 240, loss 0.03180265830098902
Epoch 250, loss 0.03114714372913371
Epoch 260, loss 0.02939308576347414
Epoch 270, loss 0.027282814114453144
Epoch 280, loss 0.0271172390261863

The loss was a little bit larger than the original model, and it trained much slower than original model.

# Using one layer of GRU for encoder and decoder

In [ ]:
class Seq2SeqEncoder4(d2l.Encoder):
    """The RNN encoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder4, self).__init__(**kwargs)
        # Embedding layer for looking up embeddings of tokens
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=dropout)

    def forward(self, X, *args):
        # The output `X` shape: (`batch_size`, `num_steps`, `embed_size`)
        X = self.embedding(X)
        # In RNN models, the first axis corresponds to time steps
        X = X.swapaxes(0, 1)
        state = self.rnn.begin_state(batch_size=X.shape[1], ctx=X.ctx)
        output, state = self.rnn(X, state)
        # `output` shape: (`num_steps`, `batch_size`, `num_hiddens`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class Seq2SeqDecoder4(d2l.Decoder):
    """The RNN decoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder4, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=dropout)
        self.dense = nn.Dense(vocab_size, flatten=False)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # The output `X` shape: (`num_steps`, `batch_size`, `embed_size`)
        X = self.embedding(X).swapaxes(0, 1)
        output, state = self.rnn(X, state)
        output = self.dense(output).swapaxes(0, 1)
        # `output` shape: (`batch_size`, `num_steps`, `vocab_size`)
        # `state[0]` shape: (`num_layers`, `batch_size`, `num_hiddens`)
        return output, state

class MaskedSoftmaxCELoss(gluon.loss.SoftmaxCELoss):
    """The softmax cross-entropy loss with masks."""
    # `pred` shape: (`batch_size`, `num_steps`, `vocab_size`)
    # `label` shape: (`batch_size`, `num_steps`)
    # `valid_len` shape: (`batch_size`,)
    def forward(self, pred, label, valid_len):
        # `weights` shape: (`batch_size`, `num_steps`, 1)
        weights = np.expand_dims(np.ones_like(label), axis=-1)
        # sequence_mask zeros out entries that are beyond the sequence lengths
        weights = npx.sequence_mask(weights, valid_len, True, axis=1)
        return super(MaskedSoftmaxCELoss, self).forward(pred, label, weights)

def train_seq2seq4(net, data_iter, lr, num_epochs, tgt_vocab, device):
    """Train a model for sequence to sequence."""
    net.initialize(init.Xavier(), force_reinit=True, ctx=device)
    trainer = gluon.Trainer(net.collect_params(), 'adam',
                            {'learning_rate': lr})
    loss = MaskedSoftmaxCELoss()
    for epoch in range(num_epochs):
        timer = d2l.Timer()
        metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
        for batch in data_iter:
            X, X_valid_len, Y, Y_valid_len = [
                x.as_in_ctx(device) for x in batch]
            bos = np.array(
                [tgt_vocab['<bos>']] * Y.shape[0], ctx=device).reshape(-1, 1)
            dec_input = np.concatenate([bos, Y[:, :-1]], 1)  # Teacher forcing
            with autograd.record():
                Y_hat, _ = net(X, dec_input, X_valid_len)
                l = loss(Y_hat, Y, Y_valid_len)
            l.backward()
            # Add gradient clipping
            d2l.grad_clipping(net, 1)
            num_tokens = Y_valid_len.sum()
            trainer.step(num_tokens)
            metric.add(l.sum(), num_tokens)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, loss {(metric[0] / metric[1])}")
    print(f'loss {metric[0] / metric[1]:.3f}, {metric[1] / timer.stop():.1f} '
          f'tokens/sec on {str(device)}')
    
def predict_seq2seq4(net, src_sentence, src_vocab, tgt_vocab, num_steps, device):
    """Predict for sequence to sequence."""
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [src_vocab['<eos>']]
    enc_valid_len = np.array([len(src_tokens)], ctx=device)
    src_tokens = d2l.truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    # Add the batch axis
    enc_X = np.expand_dims(np.array(src_tokens, ctx=device), axis=0)
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    # Add the batch axis
    dec_X = np.expand_dims(np.array([tgt_vocab['<bos>']], ctx=device), axis=0)
    output_seq = []
    for _ in range(num_steps):
        Y, dec_state = net.decoder(dec_X, dec_state)
        # We use the token with the highest prediction likelihood as the input
        # of the decoder at the next time step
        dec_X = Y.argmax(axis=2)
        pred = dec_X.squeeze(axis=0).astype('int32').item()
        # Once the end-of-sequence token is predicted, the generation of the
        # output sequence is complete
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq)),

embed_size, num_hiddens, num_layers, dropout = 32, 32, 1, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 300, d2l.try_gpu()

train_iter, src_vocab, tgt_vocab = d2l.load_data_nmt(batch_size, num_steps)
encoder = Seq2SeqEncoder4(len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqDecoder4(len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)
train_seq2seq4(net, train_iter, lr, num_epochs, tgt_vocab, device)

# Generate some sequences!
engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
for eng in engs:
    translation = predict_seq2seq4(net, eng, src_vocab, tgt_vocab, num_steps, device)
    print(f'{eng} => {translation}')

Epoch 10, loss 0.21543541840332223
Epoch 20, loss 0.1750390249084929
Epoch 30, loss 0.15615579454226083
Epoch 40, loss 0.14651427255845922
Epoch 50, loss 0.13237358800354365
Epoch 60, loss 0.12061903074190214
Epoch 70, loss 0.11149809671076955
Epoch 80, loss 0.10174629867475646
Epoch 90, loss 0.0949796431180015
Epoch 100, loss 0.08940905278613838
Epoch 110, loss 0.08453020623517478
Epoch 120, loss 0.07958375014267959
Epoch 130, loss 0.07591996123498934
Epoch 140, loss 0.07208194342338865
Epoch 150, loss 0.0674941642037624
Epoch 160, loss 0.06338788899133074
Epoch 170, loss 0.060431928908623186
Epoch 180, loss 0.056315086339224704
Epoch 190, loss 0.053811052660921434
Epoch 200, loss 0.05179602421441917
Epoch 210, loss 0.04849194147979965
Epoch 220, loss 0.04696960122695076
Epoch 230, loss 0.04410491164206332
Epoch 240, loss 0.04166348737696408
Epoch 250, loss 0.04006085101191053
Epoch 260, loss 0.0388812219378799
Epoch 270, loss 0.036646863880330116
Epoch 280, loss 0.03595688435938451
E

Change the num_layer from 2 to 1, the loss will increase and it will train faster!